In [4]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import math
%matplotlib inline

In [5]:
filepath = "google-10000-english/google-10000-english-no-swears.txt"
print(open(filepath))

import ast
filepath = "google-10000-english/google-10000-english-no-swears.txt"
words_set = pd.read_csv(filepath, names=['words'])
words_set = words_set.iloc[:]
words_set = words_set.dropna()
words_set = words_set.drop(1820)
words_set.head()

<_io.TextIOWrapper name='google-10000-english/google-10000-english-no-swears.txt' mode='r' encoding='cp1252'>


,words
0,the
1,of
2,and
3,to
4,a


In [6]:
#find the optimal frequency to guess words fastest and using the fewest steps
#changing frequency through a rotation and seeing the output and making choices based on it
#finding effectiveness by taking the Guess Success Ratio (GSR) -> guesses/UNIQUE letters in word (try 'school' to see why)
#lowest GSR "wins"
#three trials per Frequency Model on a set of 500 words randomly selected from the 10,000 word set
#first, build one operational model 
#then, transform key aspects to variables to see how effective things are
#finally, test data and wait to see what happens
#let's begin!


In [17]:
def test_runner(slope, intercept): #this function runs the various tests of the program to determine the GSR of the Frequency Model
    #UTILITY FUNCTIONS
    def init_freq_model(slope, intercept):
        #create words_details
        words_details = words_set.iloc[:]
        word_deet_len = len(words_details)
        words_details['word freq'] = np.array([intercept*word_deet_len + slope*i for i in range(word_deet_len)])
        words_details['word length'] = np.array([len(words_set.iloc[i][0]) for i in range(word_deet_len)])
        words_details.head()
        return words_details
    
    def key_of_max_val(dict_in):
        """
        >>> test_dict = {'a': 1, 'b': 2, 'c':5, 'd': 3}
        >>> print(key_of_max_val(test_dict))
        c """
        #this function returns the key that has the highest value in a dictionary
        max_val = [key for key in dict_in if dict_in[key] ==  max([dict_in[key] for key in dict_in])  ]
        return max_val
    
    #interval -> at what percent to print
    #obj_size -> how big the object is
    #curr_pos -> how far through the object calculations are
    #addendum -> the string to print after 'n% completed'
    def completion_updater(interval, obj_size, curr_pos, addendum):
        if curr_pos % (obj_size//interval) == 0 or curr_pos == obj_size:
            completion = (curr_pos*100)//obj_size
            print("{0:3d}% completed".format(completion), addendum)
            
    #custom exception for this program 
    class WordNotInDictionaryError(Exception):
        def __init__(self, value):
            self.value = value
        def __str__(self):
            print("Uh oh! Looks like you've tried to put in a word that's " + 
                  "not in our dictionary. Double check the spelling just in case, " + 
                  "but keep in mind that our dictionary is limited to only 10,000 words, " +
                  "so we may just not have that word quite yet!")
            return repr(self.value)
        
#end utility functions
#-------------------------------------------------------------------------------------------------------------------------------#

    #function that handles guessing the word and returns the number of guesses made     
    # hangman is the word we're trying to find, need to eventually replace with user input
    def guess_word(hangman, blanks=None, possible_words=pd.DataFrame(), guesses=0): 

        # possible_words: a set of words of length n 
        # blanks: a set of known letters for the words
        # objective: to find a new set of words that meets the requirements outlined in blanks
        def get_word_options(possible_words):  
            blank_dict = {}
            # create a dictionary for blanks that states the value at the index; if none, then nothing is put
            for a in range(len(blanks)):
                if blanks[a] != "":
                    blank_dict[a] = blanks[a]
            #print(blank_dict)

            #see what words meet blanks' requirements
            col_list = ["word", "word freq"]
            word_list = []
            word_freq_list = []
            for i in range(len(possible_words)): # for each word in possible_words
                word = possible_words.iloc[i][0]
                word_freq = possible_words.iloc[i][1]
                meets_req = True
                for jndex in blank_dict: # for each index in blank_dict 
                    if word[jndex] != blank_dict[jndex]:
                        meets_req = False
                if meets_req:
                    word_list.append(word)
                    word_freq_list.append(word_freq)
            word_ops = pd.DataFrame(data = {"word": np.array(word_list), "word_freq": np.array(word_freq_list)})
            return word_ops

        # create a dictionary that has the frequency of each letter from the word_options to determine the optimal guesses 
        def get_guess(word, word_ops):
            assert isinstance(word, list), "the incoming word must be a list"
            letter_dict = {}
            for i in range(len(word_ops)):
                curr_word = word_ops.iloc[i][0]
                word_freq = word_ops.iloc[i][1]
                for j in range(len(curr_word)):
                    curr_letter = curr_word[j]
                    if letter_dict.get(curr_letter, 0) == 0:
                        letter_dict[curr_letter] = 0
                    letter_dict[curr_letter] = letter_dict[curr_letter] + word_freq
            for letter in blanks:
                if letter:
                    letter_dict[letter] = -1
            return letter_dict

        # purpose: to guess a letter for the word
        # method: guess the letter with the highest frequency from the dict; if that doesn't work, guess the next highest, recursively.
        def make_guess(dict_in):
            nonlocal guesses
            guesses += 1 #increment the number of guesses thus far made
            curr_guess = key_of_max_val(dict_in)[0]
            if dict_in[curr_guess] == -1:
                raise WordNotInDictionaryError("We're unable to determine your word, but here's what we have so far: " + str(guess_dict))
            correct, word = guess_letter(curr_guess, blanks)
            if not(correct):
                dict_in[curr_guess] = -1
                return make_guess(dict_in)
            else:
                return word

        # this method returns whether the guessed letter is in the word, and the thus-filled word after the guess
        def guess_letter(letter, word):
            in_hangman = False
            if letter in hangman:
                in_hangman = True
                indices = []
                for i in range(len(hangman)):
                    if letter == hangman[i]:
                        indices.append(i)
                for i in indices:
                    word[i] = letter
            return in_hangman, word

        #see if the guessed word is the same as the word input to the program by inspecting it letter-by-letter
        def test_success(word, hangman):
            for i in range(len(hangman)):
                if word[i] != hangman[i]:
                    return False
            return True

        #next 4 lines only happen in the first iteration
        if not blanks: # this is a baseline for the parts of the word we've guessed so far
            blanks = [""]*len(hangman)
        if possible_words.empty: #this a baseline for the words that hangman could be
            possible_words = words_details[words_details['word length'] == len(blanks)][:]

        word_options = get_word_options(possible_words)
        guess_dict = get_guess(blanks, word_options)
        word = make_guess(guess_dict)
        if test_success(word, hangman):
            return guesses
        else:
            return guess_word(hangman, word, word_options, guesses)
    
    #samples the words_details to get a list of 500 random words to test
    def get_random_sample(num_samples=500):
        sampled_words_df = words_details.sample(n=num_samples)
        print(num_samples, "words generated to test")
        return [sampled_words_df.iloc[i][0] for i in range(num_samples)]
    
    #central method code. this does the testing for 500 random words 5 times
    start_time = datetime.datetime.now()
    print("start at", start_time) #start time
    words_details = init_freq_model(slope, intercept)
    reruns, sampling_size = 5, 500
    gsr_list_onesetting = []
    for i in range(1, reruns+1):
        gsr_list_onerun = []
        rand_word_list = get_random_sample(sampling_size)
        rand_word_list_len = len(rand_word_list)
        print("RUNNING TEST #" + str(i))
        count = 0
        for j in range(rand_word_list_len):
            completion_updater(10, sampling_size, j, "(" + str(datetime.datetime.now()) + ")") # print the % done and time
            word = rand_word_list[j]
            word_len = len(word)
            guesses = guess_word(word)
            gsr = guesses/word_len*100             #MAYBE NEED TO REMOVE THE *100            #MAYBE NEED TO REMOVE THE *100       #MAYBE NEED TO REMOVE THE *100
            gsr_list_onerun.append(gsr)
        av_gsr_onerun = sum(gsr_list_onerun)/float(len(gsr_list_onerun))
        print("TEST {0} GSR: {1:7.4} computed in {2:10.5}".format(i, av_gsr_onerun, str(datetime.datetime.now() - start_time))) 
        gsr_list_onesetting.append(av_gsr_onerun)
    av_gsr_onesetting = sum(gsr_list_onesetting)/float(len(gsr_list_onesetting))
    end_time = datetime.datetime.now()
    print("end at", end_time) #start time
    print("TOTAL TIME ELAPSED", end_time - start_time)
    print("GSR FOR FREQ MODEL S= " + str(round(slope, 5)) + ",I=" + str(round(intercept, 5)) + ": " + str(round(av_gsr_onesetting, 5)))
    print('\n\n\n\n')
    return av_gsr_onesetting
        #ALTERNATE CODE (less testing capabilities): gsr_one_test = [guess_word(word)/len(word) for word in rand_word_list]

In [ ]:
class XYZ:
    gsr = ()
    def __init__(self, x, y):
        self.slope = x 
        self.intercept = y
    
    def get_gsr(self):
        if not self.has_gsr():
            #print("testing for gsr!")
            gsr = test_values(self.slope, self.intercept)
            self.update_gsr(gsr)
        
    def update_gsr(self, gsr):
        self.gsr = gsr
        #print("gsr set to:", gsr)
        
    def has_gsr(self):
        return self.gsr != ()
    
    def del_gsr(self, base):
        if not(self.has_gsr()): #when self missing gsr
            #print("getting self gsr!") 
            self.get_gsr()
        if not(base.has_gsr()): #when other missing gsr
            #print("getting other gsr!")
            base.get_gsr()
        #when both have gsr, return the difference between other and self
        #print("gsr difference:", base.gsr - self.gsr, " = base.gsr (" + str(base.gsr) + ") - self.gsr (" + str(self.gsr) + ")")
        return base.gsr - self.gsr
            
    def del_pos_sum(self, base):
        #print("getting positional sum")
        #print("intercepts:", self.intercept, base.intercept)
        #print("slope:", self.slope, base.slope)
        return (self.intercept - base.intercept) + (self.slope - base.slope)
    
    def del_pos_vector(self, base):
        #print("creating positional difference vector!")
        #print([base.intercept - self.intercept, base.slope - self.slope])
        return [base.slope - self.slope, base.intercept - self.intercept]
    
    def differential(self, base):
        del_gsr = self.del_gsr(base)
        del_pos = self.del_pos_sum(base)
        #print("del_gsr:", del_gsr, "del_pos", del_pos)
        diff = del_gsr/del_pos
        #print("gsr differential:", diff)
        return diff
        
    def delta_vector(self, p1, p2):
        vector1 = self.del_pos_vector(p1)
        #print("vector for b", vector1)
        vector2 = self.del_pos_vector(p2)
        #print("vector for c", vector2)
        differential1 = p1.differential(self)
        #print("differential for b", differential1)
        differential2 = p2.differential(self)
        #print("differential for c", differential2)
        vector = [0, 0]
        vector[0] = round(differential1*vector1[0] + differential2*vector2[0], 5)
        vector[1] = round(differential1*vector1[1] + differential2*vector2[1], 5)
        #print("change vector:", vector)
        return vector

    #create secondary and tertiary points
    def get_points(self): # self is the starting point
        #change_factor = math.sqrt(abs(self.gsr-3))
        change_factor = 1/(0.1 + 4*pow(0.7, self.gsr))-0.244 
        b = XYZ(self.slope + change_factor, self.intercept)
        c = XYZ(self.slope, self.intercept + change_factor)
        b.get_gsr()
        c.get_gsr()
        #print(b)
        #print(c)
        #print("change_factor", change_factor)
        return b, c
    
    def create_new_base(self):
        b, c = self.get_points()
        coord_change_values = self.delta_vector(b, c)
        curr_coord = [self.slope, self.intercept]
        new_coord = [curr_coord[0]+coord_change_values[0], curr_coord[1]+coord_change_values[1]]
        return XYZ(new_coord[0], new_coord[1])
    
    def __repr__(self):
        slope_and_int = "Slope: " + str(round(self.slope, 5)) + "  Intercept: " + str(round(self.intercept, 5))
        if self.has_gsr():
            return slope_and_int + "  GSR: " +  str(round(self.gsr, 5))
        return slope_and_int
            
    def __str__(self):
        return self.__repr__()
        
#------------------------------------------------------------------------------------------------------------------------------#
def test_values(slope, intercept):
    ##   a = slope*b + intercept   ##
    # optimal values are: 
    return test_runner(slope, intercept)
    
def optimize_slope_and_intercept():
    print("Beginning Optimization Procedures!")
    base = XYZ(-1, 1) 
    base.get_gsr()
    margin_of_error = 0.1
    run_count = 0
    while base.gsr > margin_of_error+1 and run_count < 20:
        print("\nnew base", base)
        base = base.create_new_base()
        base.get_gsr()
        run_count += 3
    print("--------------------------------")
    print("final base:", base)
    print("run_count", run_count)
    return base, run_count

start_time = datetime.datetime.now()
final_base, computations = optimize_slope_and_intercept()
end_time = atetime.datetime.now()
print('\n\n\n\n')
print("Optimal Solution:", final_base)
print("computed in", run_count, "iterations.")
print("Total Time Elapsed:", end_time - start_time, "(from " + start_time + " to " + end_time + ")")

Beginning Optimization Procedures!
start at 2019-01-15 09:40:47.281852
running test 1
500 words generated to test
  0% completed with test 1 at 2019-01-15 09:40:48.484057
 10% completed with test 1 at 2019-01-15 09:41:52.525075
 20% completed with test 1 at 2019-01-15 09:42:53.234649
 30% completed with test 1 at 2019-01-15 09:43:49.218841
 40% completed with test 1 at 2019-01-15 09:44:41.192878
 50% completed with test 1 at 2019-01-15 09:45:38.828644
 60% completed with test 1 at 2019-01-15 09:46:37.577943
 70% completed with test 1 at 2019-01-15 09:47:36.848104
 80% completed with test 1 at 2019-01-15 09:48:36.823172
 90% completed with test 1 at 2019-01-15 09:49:27.064052
 99% completed with test 1 at 2019-01-15 09:50:24.244496
test gsr: 2.1e+02 computed in 0:09:     
running test 2
500 words generated to test
  0% completed with test 2 at 2019-01-15 09:50:25.776320
 10% completed with test 2 at 2019-01-15 09:51:18.314526
 20% completed with test 2 at 2019-01-15 09:52:17.901256
 30%

 10% completed with test 2 at 2019-01-15 11:27:10.423567
 20% completed with test 2 at 2019-01-15 11:27:57.957336
 30% completed with test 2 at 2019-01-15 11:28:42.462693
 40% completed with test 2 at 2019-01-15 11:29:24.757455
 50% completed with test 2 at 2019-01-15 11:30:10.803280
 60% completed with test 2 at 2019-01-15 11:30:56.658001
 70% completed with test 2 at 2019-01-15 11:31:46.220431
 80% completed with test 2 at 2019-01-15 11:32:34.457871
 90% completed with test 2 at 2019-01-15 11:33:22.926729
 99% completed with test 2 at 2019-01-15 11:34:04.615651
test gsr: 1.9e+02 computed in 0:15:     
running test 3
500 words generated to test
  0% completed with test 3 at 2019-01-15 11:34:05.665277
 10% completed with test 3 at 2019-01-15 11:34:47.865955
 20% completed with test 3 at 2019-01-15 11:35:34.213316
 30% completed with test 3 at 2019-01-15 11:36:20.321985
 40% completed with test 3 at 2019-01-15 11:37:06.091874
 50% completed with test 3 at 2019-01-15 11:37:53.783580
 60%

 20% completed with test 3 at 2019-01-15 12:53:30.771396
 30% completed with test 3 at 2019-01-15 12:54:19.338455
 40% completed with test 3 at 2019-01-15 12:55:07.109853
 50% completed with test 3 at 2019-01-15 12:55:48.948166
 60% completed with test 3 at 2019-01-15 12:56:35.934081
 70% completed with test 3 at 2019-01-15 12:57:25.251790
 80% completed with test 3 at 2019-01-15 12:58:13.709700
 90% completed with test 3 at 2019-01-15 12:59:02.265777
 99% completed with test 3 at 2019-01-15 12:59:43.365045
test gsr:  2e+02 computed in 0:23:     
running test 4
500 words generated to test
  0% completed with test 4 at 2019-01-15 12:59:44.668122
 10% completed with test 4 at 2019-01-15 13:00:33.098122
 20% completed with test 4 at 2019-01-15 13:01:18.987205
 30% completed with test 4 at 2019-01-15 13:02:05.344655
 40% completed with test 4 at 2019-01-15 13:02:55.431595
 50% completed with test 4 at 2019-01-15 13:03:51.307572
 60% completed with test 4 at 2019-01-15 13:04:48.330968
 70% 

 40% completed with test 4 at 2019-01-16 05:49:02.833020
 50% completed with test 4 at 2019-01-16 05:50:16.227626
 60% completed with test 4 at 2019-01-16 05:51:26.547879
 70% completed with test 4 at 2019-01-16 05:52:22.879151
 80% completed with test 4 at 2019-01-16 05:53:27.103291
 90% completed with test 4 at 2019-01-16 05:54:19.071263
 99% completed with test 4 at 2019-01-16 05:55:06.691838
test gsr: 2.1e+02 computed in 0:44:     
running test 5
500 words generated to test
  0% completed with test 5 at 2019-01-16 05:55:07.874675
 10% completed with test 5 at 2019-01-16 05:55:53.433761
 20% completed with test 5 at 2019-01-16 05:56:53.721405
 30% completed with test 5 at 2019-01-16 05:58:04.577520
 40% completed with test 5 at 2019-01-16 05:59:05.254039
 50% completed with test 5 at 2019-01-16 06:00:00.851642
 60% completed with test 5 at 2019-01-16 06:01:12.336907
 70% completed with test 5 at 2019-01-16 06:02:06.777749
 80% completed with test 5 at 2019-01-16 06:03:03.658815
 90%

In [ ]:
"""          
                 a   b
                     d   e
                 c       g
                     f          
                                       
                                               
                                                    •
                        
                        
first, we take the three points as our starters
we find the rate of change between them, (∆GSR/∆slope or ∆intercept)
move in the direction of the change between points times the rates of change
a(0,0) = 10
b(1,0) = 9
c(0,1) = 9.5
∆ab = 1/1   = 1               #a - b
∆ac = 0.5/1 = -0.5           #a - c
(b-a)*∆ab = <1,0>*1   = <1,0>
(c-a)*∆ac = <0,1>*0.5 = <0,0.5>
d(1,0.5) = 8.5
e(2,0.5) = 7.5
f(1,1.5) = 8
∆de = 1/1   = 1
∆df = 0.5/1 = 0.5
(e-d)*∆de = <1,0>*1   = <1,0>
(f-d)*∆df = <0,1>*0.5 = <0,0.5>
<1,0> + <0,0.5> = <1,0.5>
<1,0.5>+<1,0.5> = <2,1>
g(2,1)
...
...
...



"""

In [ ]:
"""def test_values(slope, intercept):
    ##   a = slope*b + intercept   ##
    # optimal values are: slope=6.2, intercept=7.5
    return abs((slope-6.2)) + abs((intercept-7.5))
    
class XYZ:
    def __init__(self, slope, intercept, gsr):
        self.slope = slope
        self.intercept = intercept
        self.gsr = gsr
    
def optimize():
    run_count = 0
    def optimize_slope(min_slope, max_slope, intercept):
        nonlocal run_count
        mean_slope, gsr_dict = (min_slope + max_slope)/2, {} 
        slope_dict = {"min_slope": min_slope, "max_slope": max_slope}  #, "mean_slope": mean_slope}
        for key in slope_dict:
            slope = slope_dict[key]
            if slope in output_dict:
                gsr_dict[slope] = output_dict[slope]
            else:
                run_count += 1
                gsr_dict[slope] = test_values(slope, intercept)
        print("min_slope:", slope_dict["min_slope"], "gsr_min:", gsr_dict[slope_dict["min_slope"]])
        print("max_slope:", slope_dict["max_slope"], "gsr_max:", gsr_dict[slope_dict["max_slope"]])
        #print("mean_slope:",slope_dict["mean_slope"],"gsr_mean:",gsr_dict[slope_dict["mean_slope"]])
        #gsr_dict = {min_slope: gsr_min, max_slope: gsr_max, mean_slope: gsr_mean}
        return gsr_dict

    def optimize_slope(min_int, max_int, slope):
        nonlocal run_count
        mean_int, gsr_dict = (min_int + max_int)/2, {} 
        slope_dict = {"min_int": min_int, "max_int": max_int}     #, "mean_int": mean_int}
        for key in slope_dict:
            slope = slope_dict[key]
            if slope in output_dict:
                gsr_dict[slope] = output_dict[slope]
            else:
                run_count += 1
                gsr_dict[slope] = test_values(slope, intercept)
        print("min_int:", slope_dict["min_int"], "gsr_min:", gsr_dict[slope_dict["min_int"]])
        print("max_int:", slope_dict["max_int"], "gsr_max:", gsr_dict[slope_dict["max_int"]])
        #print("mean_int:",slope_dict["mean_int"],"gsr_mean:",gsr_dict[slope_dict["mean_int"]])
        #gsr_dict = {min_int: gsr_min, max_int: gsr_max, mean_int: gsr_mean}
        return gsr_dict
    
    def remove_and_print_biggest(dict_in):
        print( "biggest:", dict_in.pop([key for key in dict_in if dict_in[key] ==  max([dict_in[key] for key in dict_in])  ][0]) )
    
    def key_of_fn_val(dict_in, fn):
        return [key for key in dict_in if dict_in[key] ==  fn([dict_in[key] for key in dict_in])  ][0]
    
    def update_dict_from_another(dict_to, dict_from):
        for key in dict_from:                   # for each key in the dict we're copying from
            if key not in dict_to:              # if they key isn't in the dictionary we're copying to
                dict_to[key] = dict_from[key]   # copy the key-value pair to the other dictionary
    
    def return_statement(dict_in):    
        min_val_key = [key for key in dict_in if dict_in[key] ==  min([dict_in[key] for key in dict_in])  ][0]
        smallest_output = min([dict_in[key] for key in dict_in])
        biggest_output = max([dict_in[key] for key in dict_in])
        print("run_count:", run_count)
        return min_val_key, smallest_output, biggest_output-smallest_output, dict_in
    
    margin_of_error, intercept = 0.1, 5.5
    dict_in, output_dict = {0:50, 10:100, 61:715}, {} #starting guesses for dictionary (values useless)
    while max([dict_in[key] for key in dict_in]) - min([dict_in[key] for key in dict_in]) > margin_of_error:
        remove_and_print_biggest(dict_in)                                   #remove the least accurate guess
        min_val_key = key_of_fn_val(dict_in, min)                           #get the key with the smallest value
        max_val_key = key_of_fn_val(dict_in, max)                           #get the key with the largest value
        dict_in = optimize_slope(min_val_key, max_val_key, intercept)       #test the new values
        update_dict_from_another(output_dict, dict_in)                      #update memoized dictionary from the slope/gsr dict
        print("memoized dictionary:", output_dict)                          #print memoized dictionary
        print("new_updates:", dict_in)                                      #print slope/gsr dictionary
        
    return return_statement(dict_in)

#make note of when the answer is very close to the boundary, means boundary change is needed
print('Begin!')        
optimize()
"""